#  CS 247 : Advanced Data Mining Learning
## Homework 1

### Due: 11:59 pm 01/14

##### Please read the Homework Guidance (uploaded to Canvas) carefully and make sure you fulfill all the requirements.

__Name__: Parth Shettiwar

__UID__: 105710622

## Problem 1: Multinomial Naive Bayes (50 pts)

Consider the Multinomial Naive Bayes introduced in [lecture 02 - Probabilistic Classifiers](http://web.cs.ucla.edu/~yzsun/classes/2022Winter_CS247/Slides/02NaiveBayes_LR.pdf) (please refer to page 10-23). In this problem, you are going to understand the derivations of Multinomial Naive Bayes and apply it on a real-world classification task.

### Part 1: Parameter Derivation (10 pts)

Show the derivation process to obtain the solution of the MLE estimator $\pi$. Please use the same notations as in the lecture slides.

__Hint__: The solution of $\pi$ is given on slide P23.

---



![pic](https://drive.google.com/uc?export=view&id=1eDMjB6P1Qa9HBWZXXYwBOYWOje6ryHSQ)
![pic](https://drive.google.com/uc?export=view&id=1C6t3LEXQjmqoF45AuHSVKGQsJp_Sf1At)


### Part 2: Text Classification (40 pts)

In this part, you are going to apply the multinomial naive bayes method learned in the lecture on a real-world sentiment classification dataset. 

We've provided a general framework for you, please fill all the ''TODO'' slots.

#### Part 2.1: Sklearn Implementation (5 pts)

In this part, you are going to work on the text classification task using the multinomial naive bayes function __MultinomialNB__ implemented in the sklearn library. We've provided the data processing parts, please implememt the code for training and testing, and get the probability result using ***pred_proba*** and ***pred_log_proba***.


__Hint__: 

1. You can refer to https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html to get familiar with sklearn MultinomialNB function.

2. To get the predicted results, instead of directly using the ***predict*** function in MultinomialNB, please use <u>***np.argmax(..., axis=1)***</u>, and the ... part should be the probability result obtained from ***pred_proba*** or ***pred_log_proba***.


In [1]:
# load dataset, split train and test 

from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'soc.religion.christian',  'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_test  = fetch_20newsgroups(subset='test',  categories=categories, shuffle=True, random_state=42)



In [2]:
# data processing, turn the loaded data into array

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

count_vect = CountVectorizer().fit(twenty_train['data'] + twenty_test['data']) 
X_train_feature = count_vect.transform(twenty_train['data']).toarray()
X_test_feature  = count_vect.transform(twenty_test['data']).toarray()


In [3]:
# train and test with MultinomialNB

from sklearn.naive_bayes import MultinomialNB
import numpy as np

'''
    TO DO: 
    Please implement train and test using sklearn MultinomialNB.
    You are expected to get the probability result using "pred_proba" and "pred_log_proba".
'''

def train(trn):
  
  trn.fit(X_train_feature, twenty_train["target"])
  return trn

def test(trn):

  prob_results_wo_log = trn.predict_proba(X_test_feature)
  prob_results = trn.predict_log_proba(X_test_feature)

  return prob_results, prob_results_wo_log

trn= MultinomialNB()
trained = train(trn)
prob_results_sk,prob_results_wo_log_sk = test(trained)


#### Part 2.2: Your Multinomial Naive Bayes Implementation (20 pts = 5 + 5 + 5 + 5)

In this part, you are going to implement multinomial naive bayes function by your self, then use your own multinomial naive bayes function to finish the sentimate classification task, and get the probability result using ***predict_proba_without_log*** and ***predict_proba_with_log***.

Hint: Similar to Part 1, to get the predicted results, please use <u>***np.argmax(..., axis=1)***</u>, and the ... part should be the probability result obtained from ***predict_proba_without_log*** and ***predict_proba_with_log***.


In [4]:
# My Multinomial Naive Bayes Function

class My_MultinomialNB():
    """
    Multinomial Naive Bayes
    ==========  
    Parameters
    ----------
    alpha : float, optional (default=1.0)
        Additive (Laplace/Lidstone) smoothing parameter
        (0 for no smoothing).
    """

    def __init__(self, alpha=1):
        self.alpha = alpha
        

    def fit(self, X, y):
#         Given feature X and label y, calculate beta and pi with a smoothing parameter alpha (laplace smoothing)
        self.class_indicator = {}
        for i, c in enumerate(np.unique(y)):
            self.class_indicator[c] = i
        self.n_class = len(self.class_indicator)
        self.n_feats = np.shape(X)[1]
        
        self.beta    = np.zeros((self.n_class, self.n_feats))
        self.pi      = np.zeros((self.n_class))
        '''
            TODO: 
            Please calculate self.beta and self.pi
        '''

        for i in range(self.n_class):
          self.pi[i] = np.sum(y==i)/len(y) 

        for i in range(self.n_class):
          self.beta[i,:] = (np.sum(X[(y==i)],0)+1)/(np.sum(np.sum(X[(y==i)],0),0)+self.n_feats)
           
    
    def predict_proba_without_log(self, X):
#         Given a test dataset with feature X, calculate the predicted probability of each data point
        prob = np.zeros((len(X), self.n_class))
        for i in range(self.n_class):
          prob[:,i] = np.prod(np.power(self.beta[i,:],X),1)*self.pi[i]

        prob = prob/(np.sum(prob,1).reshape(-1,1)) 
        return prob
                       
        '''
            TODO: 
            Calculate probability of which class each data belongs to, using self.beta and self.pi
        '''
 
    def predict_proba_with_log(self, X):
        log_prob = self.predict_log_proba_with_log(X)
        log_val = np.exp(log_prob - np.max(log_prob, axis=1).reshape(-1, 1))
        log_val = log_val / (np.sum(log_val,1).reshape(-1,1))
        return log_val
    
    def predict_log_proba_with_log(self, X):
#         Given a test dataset with feature X, calculate the log probability of each data point
        log_prob = np.zeros((len(X), self.n_class))
        '''
            TODO: 
            Calculate log-probability of which class each data belongs to, using self.log_beta and self.log_pi
        '''
        self.log_beta = np.log(self.beta)
        self.log_pi = np.log(self.pi)
      
        log_prob = np.matmul(X,np.transpose(self.log_beta)) + self.log_pi
      
        return log_prob


In [5]:
# train and test with My_MultinomialNB


'''
    TO DO: 
    Please implement train and test using My_MultinomialNB you implemented above.
    You are expected to get the probability result using "predict_proba_without_log" and "predict_proba_with_log".
    For this part, please use the default alpha value: alpha = 1.
'''
def train(x,y,trn):
  x = x
  y = y
  trn.fit(x,y)
  return trn

def test(x,trn):

  prob_results = trn.predict_proba_with_log(x)
  prob_results_wo_log = trn.predict_proba_without_log(x)

  return prob_results, prob_results_wo_log

trn1= My_MultinomialNB()
trained1 = train(X_train_feature,twenty_train["target"],trn1)
prob_my, prob_wo_log_my = test(X_test_feature, trained1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: RuntimeWarning: invalid value encountered in true_divide


#### Part 2.3: Complare sklearn MultinomialNB and your own My_MultinomialNB (15 pts = 2 + 2 + 2 + 2 + 2 + 5)

In part 1 and part 2, you've calculated the probability of test data using ***pred_proba*** and ***pred_log_proba***, and ***predict_proba_without_log*** and ***predict_proba_with_log***. 

Please answer:
1. Compare the accuracy, are they same or not? 
2. If they are different, please try to explain the reason.

__Hint :__

The accuracy of sklearn Multinomial NB with log and My_MultinomialNB with log should be larger than $0.9$.

In [6]:
def accuracy(y_true, y_pred):
    acc = np.equal(y_true, y_pred)
    score = sum(acc)/len(acc) # calculate the percentage of the correctness
    return score

y_true = twenty_test["target"]
# accuracy of sklearn MultinomialMB without log
'''
    TO DO: compute accuracy of sklearn MultinomialNB without log and print it out
'''


y_pred1 = np.argmax(prob_results_wo_log_sk,1)
print("sklearn MultinomialNB without log accuracy = ", accuracy(y_true,y_pred1))


# accuracy of My_MultinomialMB without log
'''
    TO DO: compute accuracy of My_MultinomialNB without log and print it out
'''


y_pred2 = np.argmax(prob_wo_log_my,1)
print("My_MultinomialNB without log accuracy = ", accuracy(y_true,y_pred2))



# accuracy of sklearn MultinomialMB with log
'''
    TO DO: compute accuracy of sklearn MultinomialNB with log and print it out
'''
y_pred3 = np.argmax(prob_results_sk,1)
print("sklearn MultinomialNB with log accuracy = ", accuracy(y_true,y_pred3))

# accuracy of My_MultinomialMB with log 
'''
    TO DO: compute accuracy of My_MultinomialNB with log and print it out
'''
y_pred4 = np.argmax(prob_my,1)
print("My_MultinomialNB with log = ", accuracy(y_true,y_pred4))

sklearn MultinomialNB without log accuracy =  0.9347536617842876
My_MultinomialNB without log accuracy =  0.36684420772303594
sklearn MultinomialNB with log accuracy =  0.9347536617842876
My_MultinomialNB with log =  0.9347536617842876


#### Write Your answer here:
1)So we have printed out 4 accuracies and we observe that the My Multinomial without log gives an accuracy of 36%. Rest all accuracies are same equal to 93.47%. Hence all accuracies are not same.   
2) The reason for My Multinmial without log giving so less accuracy is due to the fact that when we dont take log of the probabilty while computing it, we basically take the product of large number of small values (betas are very small). This leads to many of the probabilties being very small and hence when we take argmax, the inbuilt numpy function is built such that when there are multiple max values, it will simply return the first occurence which has highest value. In our case, its simply that all probabilties are almost 0 (they are indeed equal to 0 since this is due to the representation power of this notebook, after some point, like 1e-1000, is simply represented as 0), hence np.argmax will return the first index in this case, which might not be correct always, and hence we observe a low accuracy. This is not the case when we take the log. 

## Problem 2:  Logistic Regression (50 pts)

Consider the Logistic Regression introduced in [lecture 02 - Probabilistic Classifiers](http://web.cs.ucla.edu/~yzsun/classes/2022Winter_CS247/Slides/02NaiveBayes_LR.pdf) (please refer to page 30-43). In this problem, you are going to understand the derivations of Logistic Regression and apply it on a real-world classification task.

### Part 1: Logistic Regression Derivations (10pts)

Write down the matrix form operation for gradient vector and Hessian matrix for logistic regression. You are expected to represent the 1st order and 2nd order derivatives in matrix/vector form, which means you should represent them in the form of multiplication/addition/subtraction of several vectors/matrices. Your representations shouldn’t include $\sum_i$.


Hint: 
1. Please refer to the derivations on slide 40-41, and turn them into matrix form operations. 
2. You can define vector p=sigmoid(dot(x, beta)) and matrix P=diag(dot(p,(1-p))).

#### Write Your answer here:
![pic](https://drive.google.com/uc?export=view&id=18xgTLDmfTslBkFNnhllh1EFUQmYkrcqU)
![pic](https://drive.google.com/uc?export=view&id=1mknDiXCrKJPY0NSgmchLK0PrF0EYw6Eg)
![pic](https://drive.google.com/uc?export=view&id=1pyEGjFbH7xWxyInyps7Srm6owDGWl9kM)

### Part 2: Sklearn Implementation (5 pts)

In this part, you are going to work on the classification task on the __breast_cancer__ dataset using the logistic regression function __LogisticRegression__ implemented in the sklearn package. We've provided the data processing parts, please implememt the code for training and testing. Please use variable name **pred_y** for your predicted test results.


__Hint__: 

1. You can refer to https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html to get familiar with the breast_cancer dataset.

2. You can refer to https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html to get familiar with sklearn LogisticRegression function.

3. You can change the **penalty** and **solver** parameters in the definition of __LogisticRegression__ to observe the effect of different penalties, and choose the one that you think is the best. 



In [7]:
# load dataset, split train and test 

from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing 
from numpy.linalg import inv
import numpy as np
import time


X, y = load_breast_cancer(return_X_y=True)
n_data, n_features = X.shape[0], X.shape[1]
ids  = np.arange(n_data)
np.random.seed(1)
np.random.shuffle(ids)
train_ids, test_ids = ids[: n_data // 2], ids[n_data // 2: ]

train_X, train_y = X[train_ids], y[train_ids]
test_X,  test_y  = X[test_ids],  y[test_ids]



In [8]:
# train and test with LogisticRegression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report 

'''
    TO DO: 
    Please implement train and test using sklearn LogisticRegression.
'''



def train(trn, X,y):
  trn2 = trn.fit(X, y)
  return trn2
def test(trn,X):
  pred_y = trn.predict(X)
  return pred_y

trn = LogisticRegression()
trained = train(trn, train_X, train_y)
pred_y = test(trained,test_X)

# print the evaluation results
print(classification_report(test_y,pred_y))


              precision    recall  f1-score   support

           0       0.97      0.94      0.96       109
           1       0.97      0.98      0.97       176

    accuracy                           0.97       285
   macro avg       0.97      0.96      0.97       285
weighted avg       0.97      0.97      0.97       285



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


### Part 3: Your LogisticRegression Implementation (35pts = 5 + 10 + 10 + 5 + 5)

In this part, you are going to implement logistic regression function by your self. 

For optimizing Logistic Regression, you should implement the matrix version (__matrix_GA_optimizer__) of Gradient Ascent algorithm, and matrix form Newton-Raphson algorithm (__matrix_Newton_optimizer__). Please use 1e-2 as your learning rate.

Then, you are going to complete the __fit__ and __predict__ function in ***My_Logistic_Regreesion***.

Finally, you are going to get the predicted results using your own ***My_Logistic_Regression*** function with the matrix version optimizer and the Newton-Raphson optimizer.

__Hint__: 

There should not be any for loop inside the __matrix_GA_optimizer__ and __matrix_Newton_optimizer__. 

There can be for loop inside the __fit__ and __predict__ function.

The accuracy of your implementation can be lower than the sklearn version but should be at least close to $0.9$. 

In [9]:
# Sigmoid function

def sigmoid(z):
    """
      TO DO:
      Please implement the sigmoid function
    """
    z = np.clip(z,-100,100)
    return 1/(1+np.exp(-z))

#  Two different Optimizers


def matrix_GA_optimizer(W, X, y):
#     Gradient Ascent (GA) optimizer implemented with matrix operations.
    '''
        TO DO: 
        Please implement the matrix version gradient ascent optimizer
        Please use 1e-2 as your learning rate.
    '''
    lr = 1e-2

    dot = np.matmul(X,np.expand_dims(W,1))
 

    p = sigmoid(dot)
    y = np.expand_dims(y,1)
    grad = np.matmul(np.transpose(X),(y-p))
    grad = np.squeeze(grad)
    W = W + lr*grad
    return W

def matrix_Newton_optimizer(W, X, y):
#     Newton's method optimizer implemented with matrix operations.
    '''
        TO DO:
        Please implement the matrix version newton-raphson optimizer
    '''

    dot = np.matmul(X,np.expand_dims(W,1))
   
    p = sigmoid(dot)
    y = np.expand_dims(y,1)
    grad = np.matmul(np.transpose(X),(y-p))
    grad = grad.squeeze()
    p = np.squeeze(p)

    mid_matrix = np.diag(p*(1-p))
    hess = np.matmul((np.matmul(np.transpose(X),mid_matrix)),X)

    W = W + np.matmul(np.linalg.pinv(hess),grad)
    return W


In [10]:
# Your own Logistic Regression Function

class My_Logistic_Regression():
#     Parameters
#     ----------
#         n_features : int, feature dimension
#         optimizer  : function, one optimizer that takes input the model weights
#                      and training data to perform one iteration of optimization.

    def __init__(self, n_features, optimizer):
        self.W = np.random.rand(n_features+1)/100
        self.optimizer = optimizer
        
    def fit(self, X, y):
        """
          TO DO:
          Please implement the fit function for X, y using the optimizer you write
        """

        curr_loss = 1
        prev_loss = 0
        iterr = 1000
        one_array = np.ones((len(X),1))
        X = (train_X - np.min(X,0))/(np.max(X,0)-np.min(X,0))
        X = np.concatenate((one_array,X),1)
        count = 0
        while(count<iterr):
          count += 1
          prev_loss = curr_loss
          curr_loss = (sum(y*np.matmul(X,self.W) - np.log(1+np.exp(np.clip(np.matmul(X,self.W),-100,100)))))
          self.W = self.optimizer(self.W,X,y)
     
    def predict(self, X):
        """
          TO DO:
          Please complete the predict function given your learned W
          Specifically, you need to compute the probability given X and W
        """
        one_array = np.ones((len(X),1))
        X = (X - np.min(X,0))/(np.max(X,0)-np.min(X,0))
        X = np.concatenate((one_array,X),1)
        return((sigmoid(np.matmul(X,self.W))>0.5))
        
        
        

In [11]:
# Finally, run the following code to train and test with your own My_Logistic_Regression function
# Compare the accuracy and running time of both optimization versions (for-loop and matrix).
for optimizer in [matrix_GA_optimizer, matrix_Newton_optimizer]:
    
    start_time = time.time()

    my_clf = My_Logistic_Regression(n_features, optimizer)    
    my_clf.fit(train_X, train_y)
    my_pred_y = my_clf.predict(test_X)
    
    end_time = time.time()
    print('Training time: %d s' % (end_time - start_time))
    print(classification_report(test_y,my_pred_y))

Training time: 0 s
              precision    recall  f1-score   support

           0       1.00      0.92      0.96       109
           1       0.95      1.00      0.98       176

    accuracy                           0.97       285
   macro avg       0.98      0.96      0.97       285
weighted avg       0.97      0.97      0.97       285

Training time: 1 s
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       109
           1       0.97      0.97      0.97       176

    accuracy                           0.96       285
   macro avg       0.96      0.96      0.96       285
weighted avg       0.96      0.96      0.96       285



## Bonus Problem: Poisson Regression (10 pts)

Consider the Generalized Linear Model (GLM) introduced in [lecture 02 - Probabilistic Classifiers](http://web.cs.ucla.edu/~yzsun/classes/2022Winter_CS247/Slides/02NaiveBayes_LR.pdf) (please refer to page 46-48).

### Part 1 Poisson Regression Derivation (2 pts)
We know that the Poisson distribution is $P(y, \lambda)=\frac{\lambda^ye^{-\lambda}}{y!}$. Let $\eta=\ln(\lambda)$. In Poisson regression, we use linear model to predict the value of $\eta$, i.e., $\eta=\mathbf{x}^T\boldsymbol{\beta}$. Please write down $P(y;\mathbf{x},\boldsymbol{\beta})$.

#### Write Your answer here:
$P(y;x,\beta)$ = $\frac{1}{y!}e^{y(\mathbf{x}^T\boldsymbol{\beta})-e^{\mathbf{x}^T\boldsymbol{\beta}}}$.  
Here η = $x^{T}β$,  
T(y) = y,  
b(y) = $\frac{1}{y!}$,  
a(η) = $e^{x^{T}β}$

### Part 2 Optimization (1 pt)
Given a training dataset $D=\{\mathbf{x}_i,y_i\}_{i=1,\cdots,n}$, please name an algorithm that can be used to learn the parameters $\boldsymbol{\beta}$. Explain your answer.

#### Write Your answer here:
We can use Linear regression with Gradient ascent optimization to learn the parameters. The function we will maximize is the standard likelihood function and we know that it is concave, hence if we simply set the derivate to 0 with respect to beta, we will aproach its maximum in gradient ascent approach.

### Part 3 Application (2 pts)
Please write down at least two real-world applications you think Poisson regression can solve. 

#### Write Your answer here:
Poisson Regression is mainly used to model count based data, where the output is usually an integral value ranging in non negative integers.  

1) We can model the number of customers who would come at a shop on a particular day.    
The features we can take are: the day number (ranging from 1 to 7 for 7 days of week), weather of that day, is the day holiday or not, number of customers coming on previous day etc.  

2) Number of people visiting a website per hour.  
Again the features can be, which hour of day is it, network load on the website domain, number of people visiting site in last few days, rating of website or product on wesbite (if someone has put a feedback form, we can get this data) etc.


### Part 4: Inference (5 pts = 2 pts + 3 pts)

We want to use Poisson regression to predict the number of storms in 4 cities this winter. $\mathbf{x}$ is the features of all the cities and $\boldsymbol{\beta}$ is already learned. 
* Please implement the code to predict the __expected number__ of storms for each city ($E(Y|\mathbf{x},\beta)$) and output your results. 
* Please implement the code to compute the probability of $P(Y\ge5;x,\beta)$ for each city and output your results.

__Hint :__

1. For Poisson distribution, $E(Y|\lambda)=\lambda$. 

2. $P(y\ge5;x,\beta)=1-P(y=0;x,\beta)-P(y=1;x,\beta)-P(y=2;x,\beta)-P(y=3;x,\beta)-P(y=4;x,\beta)$.


In [12]:
import numpy as np
X=np.array([[0.1,0.3,0.5],[2.7,1.6,0.1],[0.1,2.4,0.3],[0.6,0.7,5.0]])
beta=np.array([0.1,0.5,-0.2])

"""
 TO DO:
 Please implement the code to compute and print the P(y;x,\beta) 
"""
n = np.matmul(X,beta)
mean = np.exp(n)
print("Expected number of storms for each city = ")
print(mean)

def poisson(mean,k):
  return ((mean**k)*np.exp(-mean))/(np.math.factorial(k))
suma = 0
for i in range(5):
  suma += poisson(mean,i)
print("Probabilty that number of storms greater than equal to 5 in each city = ")
print(1-suma)  


Expected number of storms for each city = 
[1.06183655 2.85765112 3.15819291 0.55432728]
Probabilty that number of storms greater than equal to 5 in each city = 
[0.00469862 0.16141173 0.21198157 0.00027568]
